In [2]:
using ITensors, ITensorMPS, Plots, Statistics
using oDMRG

In [3]:
# Physical parameters: system size and coupling factor gamma
L = 10
gamma = 0.1

# Create sites using customized TwoSpinHalf sites
sites = siteinds("TwoSpinHalf", L)

# Physical Parameters: Jx, Jy, Jz, dissipators location, dissipators temperature, coupling vector, magnetic field
params = (
    Jx = 1.0, Jy = 1.0, Jz = 1.0,
    dissipatorsVec = [1, L],
    dissipatorsTempValues = [1.0, 0.0],
    gammaVec = [gamma, gamma],
    hVec = zeros(L)
)

# LdL MPO constructor
LdL = LdLXYZConstruct(sites, params...; maxdim=500, cutoff=1e-8)
rho = make_ivec(L, sites)


Constructing Liouvillian for a spin chain of size 10 using 2 dissipators

Constructing Liouvillian for a spin chain of size 10 using 2 dissipators


MPS
[1] ((dim=1|id=314|"Link,l=1"), (dim=4|id=653|"Site,TwoSpinHalf,n=1"))
[2] ((dim=1|id=314|"Link,l=1"), (dim=4|id=682|"Site,TwoSpinHalf,n=2"), (dim=1|id=180|"Link,l=2"))
[3] ((dim=1|id=180|"Link,l=2"), (dim=4|id=854|"Site,TwoSpinHalf,n=3"), (dim=1|id=700|"Link,l=3"))
[4] ((dim=1|id=700|"Link,l=3"), (dim=4|id=133|"Site,TwoSpinHalf,n=4"), (dim=1|id=236|"Link,l=4"))
[5] ((dim=1|id=236|"Link,l=4"), (dim=4|id=79|"Site,TwoSpinHalf,n=5"), (dim=1|id=670|"Link,l=5"))
[6] ((dim=1|id=670|"Link,l=5"), (dim=4|id=681|"Site,TwoSpinHalf,n=6"), (dim=1|id=596|"Link,l=6"))
[7] ((dim=1|id=596|"Link,l=6"), (dim=4|id=125|"Site,TwoSpinHalf,n=7"), (dim=1|id=550|"Link,l=7"))
[8] ((dim=1|id=550|"Link,l=7"), (dim=4|id=637|"Site,TwoSpinHalf,n=8"), (dim=1|id=388|"Link,l=8"))
[9] ((dim=1|id=388|"Link,l=8"), (dim=4|id=725|"Site,TwoSpinHalf,n=9"), (dim=1|id=230|"Link,l=9"))
[10] ((dim=1|id=230|"Link,l=9"), (dim=4|id=414|"Site,TwoSpinHalf,n=10"))


In [4]:
cutoff = 1E-10
# Warm up routine (many bd=1 dmrg runs)
println("Initializing warm-up routine")
energyFin = floatmax(Float64)
warmUpTag = true
convergenceThreshold = 0.0000001 #1E-10
warmUpTracker = 0
while warmUpTag
    global warmUpTracker += 1
    energyIni = energyFin;
    global energyFin, rho = dmrg(LdL,rho; nsweeps=1, maxdim=1, cutoff=cutoff, outputlevel=0)

    println("Difference between energies: ", (energyIni-energyFin))

    if abs(abs(energyIni)-abs(energyFin)) < convergenceThreshold
        global warmUpTag = false
        println("EndedWarmUpAfter ", warmUpTracker, " sweeps")
    end
end

Initializing warm-up routine
Difference between energies: 1.7976931348623157e308
Difference between energies: -2.9879968508339516e-6
Difference between energies: -4.6318155462188315e-6
Difference between energies: -4.590038308549538e-6
Difference between energies: -3.925991528319628e-6
Difference between energies: -3.1188119784530954e-6
Difference between energies: -2.368804153718429e-6
Difference between energies: -1.746733182983462e-6
Difference between energies: -1.2623014526980114e-6
Difference between energies: -8.995181168813815e-7
Difference between energies: -6.348137162603962e-7
Difference between energies: -4.4534165155596384e-7
Difference between energies: -3.118430917936621e-7
Difference between energies: -2.1815375106370993e-7
Difference between energies: -1.5243800177699995e-7
Difference between energies: -1.0657868898533707e-7
Difference between energies: -7.465909135362381e-8
EndedWarmUpAfter 17 sweeps


In [ ]:
Ivec = make_ivec(L, sites)

# Now initiating iterative maxDim increase oDMRG
finalMaxDim = 500
maxDimInc = 2;
energyFin = 10000000.0
sweepBDChangeThresholdValue = 0.0006 # good values: 0.0005 ~ 0.001
finalSweep = 2000; # usually 1000~2000 works for N = 10. Experiment with more for larger sizes
maxdim2 = 2;
for iniSweep in 1:finalSweep
    energyIni = energyFin
    global energyFin, rho = dmrg(LdL,rho; nsweeps=1, maxdim=maxdim2, cutoff=cutoff)

    # Now we calculate the flux
    currents = calculate_spinFlux(Ivec, rho, sites, [1:L;])
    if abs((energyIni - energyFin)/energyIni) < sweepBDChangeThresholdValue
        global maxdim2 += maxDimInc
    end
end
println("Final energy = $energyFin")

In [ ]:
currents = calculate_spinFlux(Ivec, rho, sites, [1:L;])
println("Analytical value of the current: ", -fluxXXX(L, gamma))
println("Numerical approximation of the current: ", mean(currents))

analyticalMag = magnonDensityXXX(L, gamma)
magDensity = calculate_magnetization(Ivec, rho, sites)

# println("Analytical Magnetization profile: ", analyticalMag)
# println("Numerical magProf Approx (real) : ", real.(magDensity))
# println("Numerical magProf Approx (cmpx) : ", imag.(magDensity))

plot([1:L;], real.(analyticalMag), linewidth=3, label = "Analytical magnetization")

# Using the imaginary part of the magnon density as  uncertainty (clearly an overestimatation)
plot!([1:L;], real.(magDensity),yerr=imag.(magDensity), linestyle=:dash, linewidth=3, title = "Magnetization Profile: XXX system", label = "oDMRGjl approximation")


